The Kaggle project requires the ranking to be optimized specifically for every display_id (i.e. batches with inconsistent size). It is possible to transform a ranking problem into a two-class classification although it will limit the optimization functions to linear ranking algorithms. For example, only linear kernel for RankSVM.

In [1]:
import pandas as pd
ev_train = pd.read_csv('not_transformed.csv')
ev_train.head()

,display_id,ad_id,clicked,landing_page,campaign_id,advertiser_id,uuid,document_id,platform,geo_location,day,hour,category_id,topic_id,entity_id,source_id,publisher_id
0,1459,132913,0,1021154,16925,3074,faf57f287517f4,1649164,2,CA,2016-06-14,4,1610.0,16.0,32e11cfe904b4f16048f0f5652606452,1296.0,777.0
1,1459,176834,0,1145458,21541,2352,faf57f287517f4,1649164,2,CA,2016-06-14,4,1610.0,16.0,32e11cfe904b4f16048f0f5652606452,1296.0,777.0
2,1459,206927,1,1523781,23817,2635,faf57f287517f4,1649164,2,CA,2016-06-14,4,1610.0,16.0,32e11cfe904b4f16048f0f5652606452,1296.0,777.0
3,1820,45305,0,941800,6310,1198,1f73468d752afe,1764575,2,IL,2016-06-14,4,1907.0,184.0,89545f9fff7cf39c6f9b098bac21eb0d,5947.0,320.0
4,1820,97384,1,785003,12638,1644,1f73468d752afe,1764575,2,IL,2016-06-14,4,1907.0,184.0,89545f9fff7cf39c6f9b098bac21eb0d,5947.0,320.0


In [2]:
dummy_list_ad = ['landing_page','campaign_id','advertiser_id']
dummy_list_other = ['category_id','entity_id','topic_id','source_id','publisher_id']
for i in dummy_list_other:
    analysis = ev_train.groupby(i)['clicked'].agg(['sum','count'])
    analysis['ratio'] = analysis['sum']*1.0/analysis['count']
    temp_list = analysis.sort_values('count',ascending=False).iloc[20:].index.values.tolist()
    ev_train[i].replace(to_replace=temp_list,value='MinorityClass',inplace=True)
for i in dummy_list_ad:
    analysis = ev_train.groupby(i)['clicked'].agg(['sum','count'])
    analysis['ratio'] = analysis['sum']*1.0/analysis['count']
    temp_list = analysis.sort_values('count',ascending=False).iloc[300:].index.values.tolist()
    ev_train[i].replace(to_replace=temp_list,value='MinorityClass',inplace=True)
ev_train.head()

,display_id,ad_id,clicked,landing_page,campaign_id,advertiser_id,uuid,document_id,platform,geo_location,day,hour,category_id,topic_id,entity_id,source_id,publisher_id
0,1459,132913,0,MinorityClass,MinorityClass,MinorityClass,faf57f287517f4,1649164,2,CA,2016-06-14,4,1610.0,16.0,MinorityClass,MinorityClass,MinorityClass
1,1459,176834,0,MinorityClass,MinorityClass,2352,faf57f287517f4,1649164,2,CA,2016-06-14,4,1610.0,16.0,MinorityClass,MinorityClass,MinorityClass
2,1459,206927,1,MinorityClass,MinorityClass,2635,faf57f287517f4,1649164,2,CA,2016-06-14,4,1610.0,16.0,MinorityClass,MinorityClass,MinorityClass
3,1820,45305,0,MinorityClass,MinorityClass,1198,1f73468d752afe,1764575,2,IL,2016-06-14,4,1907.0,184.0,MinorityClass,MinorityClass,MinorityClass
4,1820,97384,1,MinorityClass,MinorityClass,1644,1f73468d752afe,1764575,2,IL,2016-06-14,4,1907.0,184.0,MinorityClass,MinorityClass,MinorityClass


In [3]:
all_input = ev_train.copy()
drop_list=['display_id','ad_id','clicked','uuid','document_id']
for i in drop_list:
    all_input.drop(i,axis=1,inplace=True)

for i in all_input.columns:
    if all_input[i].dtype!='object':
        all_input[i] = all_input[i].astype('object')
        
all_input = pd.get_dummies(all_input)
        
input_train = all_input.ix[:25833]
input_test = all_input.ix[25834:]
target_train = ev_train.ix[:25833,'clicked']
target_test = ev_train.ix[25834:,'clicked']
index_range = ev_train.groupby('display_id').count()['ad_id'].tolist()
train_index_range = ev_train.ix[:25833].groupby('display_id').count()['ad_id'].tolist()
test_index_range = ev_train.ix[25834:].groupby('display_id').count()['ad_id'].tolist()
input_train.head()

,landing_page_129047,landing_page_156664,landing_page_208091,landing_page_257734,landing_page_331052,landing_page_361129,landing_page_371327,landing_page_393819,landing_page_428294,landing_page_466331,...,publisher_id_784.0,publisher_id_867.0,publisher_id_874.0,publisher_id_875.0,publisher_id_1042.0,publisher_id_1066.0,publisher_id_1089.0,publisher_id_1142.0,publisher_id_1228.0,publisher_id_MinorityClass
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
ev_test = pd.DataFrame(ev_train.ix[25834:][['display_id','ad_id','clicked']]).copy().reset_index().drop('index',axis=1)
def map12(y_pred):
    df_pred = pd.DataFrame(y_pred.tolist(),columns=['prob'])
    ev_test_t = pd.concat([ev_test,df_pred],axis=1)
    test_index_range = ev_test_t.groupby('display_id').count()['ad_id'].tolist()
    start = 0
    map12 = 0
    for p in test_index_range:
        end = start+p
        temp = ev_test_t.ix[start:end-1].copy().reset_index().drop('index',axis=1)
        temp2= temp.sort_values('prob',ascending=False).reset_index().drop('index',axis=1)
        index = temp2.ix[temp2['clicked']==1].index[0]
        map12 = map12+(1.0/(index+1))
        start = end
    return map12/len(test_index_range)

In [6]:
import itertools
import numpy as np
from scipy import stats
import pylab as pl
from sklearn import svm, linear_model, model_selection

k = 0
Xp, yp, diff = [], [], []
start = 0
for n in train_index_range:
    end = start + n
    for (i,j) in itertools.combinations(range(n), 2):
        if target_train[start+i] == target_train[start+j]:
            continue
        else:
            Xp.append(input_train.ix[start+i].values - input_train.ix[start+j].values)
            diff.append( target_train[start+i] -  target_train[start+j])
            yp.append(np.sign(diff[-1]))
            if yp[-1] != (-1) ** k:
                yp[-1] *= -1
                Xp[-1] *= -1
                diff[-1] *= -1
            k += 1
    start = end

In [9]:
import numpy as np
from sklearn.svm import LinearSVC
clf = LinearSVC(dual=False,C=3)
clf.fit(Xp, yp)
y_pred = clf.decision_function(input_test.values)

In [10]:
map12(y_pred)

0.5746818404307411